In [10]:
#Given a year 2009
# we have N cells 
# Each cell contains information [{sst,wind_dir, cholorphyll, index, month, month_index}] (array is sorted by month)


In [11]:
import math
import json
from pprint import pprint
import pandas as pd
from datetime import datetime

In [12]:
def drange(start, stop, step):
    r = start
    while r < stop:
        yield r
        r += step

In [13]:
def readSSTData(year):
    data = []
    filename = "../data/sstmay" + str(year) + '.json'
    with open(filename) as f:
        data = json.load(f)
    return data

In [14]:
def readCholorData(year):
    data = []
    filename = "../data/chlorophyllmay" + str(year) + '.json'
    with open(filename) as f:
        data = json.load(f)
    return data

In [15]:
def readWindData():
    data = []
    filename = "../data/wind.json"
    with open(filename) as f:
        data = json.load(f)
    return data
    

In [16]:
def initMatrix(yearRange):
    matrix = []
    smallestLat = 35.775
    largestLat = 39.15
    smallestLon = 235.5625
    largestLon = 238.9375
    latRange = largestLat - smallestLat
    lonRange = largestLon - smallestLon

    latStep = latRange/10
    lonStep = lonRange/10

    latInterval = drange(smallestLat, largestLat, latStep)
    latInc = [float("{0:.4f}".format(x)) for x in latInterval]
    latInc = latInc[:-1]

    lonInterval = drange(smallestLon, largestLon, lonStep)
    lonInc = [float("{0:.4f}".format(x)) for x in lonInterval]
    
    for lat_i, lat_ in enumerate(latInc):
        if(lat_i+1 < len(latInc)-1):
            for lon_i,lon_ in enumerate(lonInc):
                if(lon_i+1 < len(lonInc)-1):
                    latRnge =[lat_,latInc[lat_i+1]]
                    lonRnge =[lon_,lonInc[lon_i+1]]
                    cell = {'latRange': latRnge, 'lonRange': lonRnge}
                    for year in yearRange:
                        cell[year] = {'sst':-9999,'windDegree':-9999}
                    matrix.append(cell)
        
    return matrix
    

In [17]:
#extend this to handle days for that month.
# e.g sst:(float) -> sst:[float,float,...] (sorted by day)
def processFile(years,matrix):
    for year in years:
        sstData = readSSTData(year)
        rows = sstData["table"]["rows"]
        for idx, cell in enumerate(matrix):
            cell = matrix[idx]
            totalIntTemp = 0
            count = 0
            avgIntTemp = 0
            for row in rows:
                lat = row[2]
                lon = row[3]
                sst = row[4]
                if((cell['latRange'][0] <= lat <= cell['latRange'][1] ) and (cell['lonRange'][0] <= lon <= cell['lonRange'][1])):
                    if(sst != None and sst != 0):
                        count = count + 1
                        totalIntTemp = totalIntTemp + sst
            if count != 0:
                avgIntTemp = totalIntTemp/count
                cell[year]['sst'] = float("{0:.4f}".format(avgIntTemp))
        
    return matrix

In [18]:
def processWindData(matrix):
    windData = readWindData()
    windRows = windData['table']['rows']
    for idx, cell in enumerate(matrix):
        cell = matrix[idx]
        for wRow in windRows:
            datetime_object = datetime.strptime(wRow[0], '%Y-%m-%dT%H:%M:%SZ')
            year = datetime_object.year
            lat = wRow[2]
            lon = wRow[3]
            xWind = wRow[4]
            yWind = wRow[5]
            if((cell['latRange'][0] <= lat <= cell['latRange'][1] ) and (cell['lonRange'][0] <= lon <= cell['lonRange'][1])):
                if(xWind != None and yWind != None):
                    cell[year]['windDegree'] = wind_degree = math.degrees(math.atan(yWind/xWind))
    return matrix

In [24]:
def processChlorophyll(years,matrix):
    for year in years:
        cholorData = readCholorData(year)
        rows = cholorData["table"]["rows"]
        for idx, cell in enumerate(matrix):
            cell = matrix[idx]
            totalIntTemp = 0
            count = 0
            avgIntTemp = 0
            for row in rows:
                lat = row[2]
                lon = row[3]
                chloro = row[4]
                if((cell['latRange'][0] <= lat <= cell['latRange'][1] ) and (cell['lonRange'][0] <= lon <= cell['lonRange'][1])):
                    if(chloro != None ):
                        count = count + 1
                        totalIntTemp = totalIntTemp + chloro
            if count != 0:
                avgIntTemp = totalIntTemp/count
                cell[year]['chloro'] = float("{0:.4f}".format(avgIntTemp))
        
    return matrix

In [25]:
#run this function for all 
years = [2009,2010,2011,2012,2013,2014,2015,2016,2017,2018]
sstYears = [2011,2015]
matrix = initMatrix(years)
matrix = processFile(sstYears,matrix)
matrix = processWindData(matrix)
matrix = processChlorophyll(sstYears,matrix)

In [115]:
with open('data.json', 'w') as outfile:
    json.dump(matrix, outfile)

In [26]:
matrix

[{2016: {'sst': -9999, 'windDegree': -82.12673472493668},
  2017: {'sst': -9999, 'windDegree': -88.52408595720108},
  2018: {'sst': -9999, 'windDegree': -69.94635230819601},
  'lonRange': [235.5625, 235.9],
  'latRange': [35.775, 36.1125],
  2009: {'sst': -9999, 'windDegree': -49.719220258227914},
  2010: {'sst': -9999, 'windDegree': 46.16051282090083},
  2011: {'chloro': 0.2558, 'sst': 13.3142, 'windDegree': -76.93208748518201},
  2012: {'sst': -9999, 'windDegree': -50.052108988163155},
  2013: {'sst': -9999, 'windDegree': -78.01428210489628},
  2014: {'sst': -9999, 'windDegree': -78.59122351344351},
  2015: {'chloro': 0.2439, 'sst': 12.8655, 'windDegree': -51.106840220889666}},
 {2016: {'sst': -9999, 'windDegree': -83.33159572042065},
  2017: {'sst': -9999, 'windDegree': -83.62590882454896},
  2018: {'sst': -9999, 'windDegree': -68.31918749291859},
  'lonRange': [235.9, 236.2375],
  'latRange': [35.775, 36.1125],
  2009: {'sst': -9999, 'windDegree': -50.35853537700364},
  2010: {'sst